In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [90]:
fake = pd.read_csv('data/fake.csv')
true = pd.read_csv('data/true.csv')

In [91]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [92]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Data preprocessing

In [93]:
fake['label'] = 1
true['label'] = 0

In [94]:
data = pd.concat([fake, true], axis=0).reset_index(drop=True)
data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [95]:
df = data.drop(['title', 'subject', 'date'], axis=1)

In [96]:
df.tail()

,text,label
44893,BRUSSELS (Reuters) - NATO allies on Tuesday we...,0
44894,"LONDON (Reuters) - LexisNexis, a provider of l...",0
44895,MINSK (Reuters) - In the shadow of disused Sov...,0
44896,MOSCOW (Reuters) - Vatican Secretary of State ...,0
44897,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,0


In [97]:
df.drop_duplicates(inplace=True)

In [98]:
print(df.isna().sum())
print(df.duplicated().sum())

text     0
label    0
dtype: int64
0


In [99]:
from sklearn.model_selection import train_test_split

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r"@\w+|#\w+", '', text)
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"\d+", '', text)
    text = text.strip()
    return text

X_train = X_train.apply(lambda x: clean_text(x))
X_test = X_test.apply(lambda x: clean_text(x))

In [101]:
import spacy

nlp = spacy.load('en_core_web_sm')

def tokenizer(text):
    return [token.text for token in nlp.tokenizer(text)]

X_train = [tokenizer(text) for text in X_train]
X_test = [tokenizer(text) for text in X_test]

In [102]:
text_lengths = df["text"].apply(lambda x: len(x.split()))
print(np.quantile(text_lengths, 0.95))

880.0


In [103]:
from collections import Counter

def build_vocab(token_lists, min_freq=1):
    counter = Counter(token for tokens in token_lists for token in tokens)
    
    vocab = {"<pad>": 0, "<unk>": 1}
    for token, count in counter.items():
        if count >= min_freq:
            vocab[token] = len(vocab)
    return vocab

vocab = build_vocab(X_train)

def text_to_indices(tokens):
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]

train_indices = [text_to_indices(tokens) for tokens in X_train]
test_indices = [text_to_indices(tokens) for tokens in X_test]

In [104]:
max_len = 880

def pad_sequence_fixed_length(sequence, max_length):
    if len(sequence) > max_length:
        return sequence[:max_length]
    else:
        return sequence + [vocab["<pad>"]] * (max_length - len(sequence))

train_padded = [pad_sequence_fixed_length(seq, max_len) for seq in train_indices]
test_padded = [pad_sequence_fixed_length(seq, max_len) for seq in test_indices]

In [105]:
X_train = torch.tensor(train_padded, dtype=torch.long)
X_test = torch.tensor(test_padded, dtype=torch.long)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [106]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
train_dataset = TextDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Classic ML models

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)

model.fit(train_padded, y_train)
predictions = model.predict(test_padded)

accuracy_score(y_test, predictions)

/Users/andrik/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.656921086675291

In [108]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(verbose = 0)
model.fit(train_padded, y_train)
predictions = model.predict(test_padded)
accuracy_score(y_test, predictions)

/Users/andrik/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/andrik/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0.9899094437257439

In [109]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(train_padded, y_train)
predictions = model.predict(test_padded)
accuracy_score(y_test, predictions)

0.991849935316947

## GRU model

In [122]:
device = torch.device("cpu")
print(device)

class GRU_nn(nn.Module):
    def __init__(self):
        super(GRU_nn, self).__init__()
        self.embedding = nn.Embedding(len(vocab), 100)
        self.gru = nn.GRU(100, 50, 1, batch_first=True)
        self.fc1 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.embedding(x.long())
        x, _ = self.gru(x)
        x = x[:, -1, :]
        x = torch.sigmoid(self.fc1(x))
        return x
    

model = GRU_nn().to(device)
model.train()
crit = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

## for epochs in range(5):
##    for X_batch, y_batch in train_loader:
##        X_batch = X_batch.to(device)
##        y_batch = y_batch.to(device)
##        
##        y_hat = model(X_batch)
##        loss = crit(y_hat, y_batch)
##        loss.backward()
##        optimizer.step()
##        optimizer.zero_grad()
##    print(f"Epoch: {epochs+1}, loss: {loss}")

cpu


In [123]:
model.load_state_dict(torch.load("gru.pth", map_location=torch.device('cpu'), weights_only=True))
model.eval()

GRU_nn(
  (embedding): Embedding(173907, 100)
  (gru): GRU(100, 50, batch_first=True)
  (fc1): Linear(in_features=50, out_features=1, bias=True)
)

In [125]:
test_dataset = TextDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_preds = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        prediction = model(X_batch)
        all_preds.append(prediction.cpu())
all_preds = torch.cat(all_preds)
predicted_classes = (all_preds > 0.5).int()
print("Test Accuracy:", accuracy_score(y_test, predicted_classes))

Test Accuracy: 0.9975420439844761


## CNN model

In [134]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        self.embedding = nn.Embedding(len(vocab), 100)
        self.conv1 = nn.Conv1d(100, 50, 2)
        self.conv2 = nn.Conv1d(50, 25, 2)
        self.fc1 = nn.Linear(25*878, 1)

    def forward(self, x):
        x = self.embedding(x.long())
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.shape[0], -1)
        x = torch.sigmoid(self.fc1(x))
        return x
    
model2 = CNN_model().to(device)
optimizer = optim.Adam(model2.parameters(), lr = 0.001)

model2.train()

## for epochs in range(5):
##    for X_batch, y_batch in train_loader:
##        X_batch = X_batch.to(device)
##        y_batch = y_batch.to(device)
##        
##        y_hat = model2(X_batch)
##        loss = crit(y_hat, y_batch)
##        loss.backward()
##        optimizer.step()
##        optimizer.zero_grad()
##    print(f"Epoch: {epochs+1}, loss: {loss}")

CNN_model(
  (embedding): Embedding(173907, 100)
  (conv1): Conv1d(100, 50, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(50, 25, kernel_size=(2,), stride=(1,))
  (fc1): Linear(in_features=21950, out_features=1, bias=True)
)

In [135]:
model2.load_state_dict(torch.load("cnn.pth", map_location=torch.device('cpu'), weights_only=True))
model2.eval()

CNN_model(
  (embedding): Embedding(173907, 100)
  (conv1): Conv1d(100, 50, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(50, 25, kernel_size=(2,), stride=(1,))
  (fc1): Linear(in_features=21950, out_features=1, bias=True)
)

In [136]:
all_preds = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        prediction = model2(X_batch)
        all_preds.append(prediction.cpu())
all_preds = torch.cat(all_preds)
predicted_classes = (all_preds > 0.5).int()
print("Test Accuracy:", accuracy_score(y_test, predicted_classes))

Test Accuracy: 0.9980595084087969
